In [7]:
import requests
from bs4 import BeautifulSoup

In [8]:
class BillFinder:
    
    def __init__(self):
        
        self.source_urls = {'Canada': 'https://en.wikipedia.org/wiki/List_of_Canadian_electric_utilities',
                            'USA': 'https://en.wikipedia.org/wiki/List_of_United_States_electric_companies',
                            'UK': 'https://www.utilitysavingexpert.com/energy/suppliers/'}
    
    def get_energy_companies(self, country: str = None):
    
        company_set = set()
        soup = BeautifulSoup(requests.get(self.source_urls[country]).text)
        
        if country == 'Canada':
            for t in soup.find_all('table', role="presentation"):
                for _ in t.find_all('li'):
                    for a in _.find_all('a'):
                        company_set.add(a.text)
        elif country == 'USA':
            for t in soup.find('span', id="List_of_US_electric_companies_by_state"):
                for e in t.parent.next_elements:
                    if e.name == 'table':
                        break
                    if e.name == 'ul':
                        for _ in e.find_all('li'):
                            if company_name := _.text.strip():
                                company_set.add(company_name)
        elif country == 'UK':
            for header_text in ['List of UK energy suppliers', 'List of business energy suppliers']:
                p = soup.find('h2', string=header_text).parent
                for _ in p.find_all('div', class_="table-responsive"):
                    for row in _.find_all('tr'):
                        if tds := row.find_all('td'):
                            company_set.add(tds[1].text)
                            
        print(f'found {len(company_set):,} companies')
                        
        return company_set

In [9]:
bf = BillFinder()

In [ ]:
bf.get_energy_companies(country='UK')